In [46]:
# MULTI-MODAL TSP TRAVEL PLANNER

from dotenv import load_dotenv
import os
import requests
import googlemaps
import time
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2 # OR-Tools TSP solver

load_dotenv()

maps_api_key = os.getenv("MAPS_API_KEY")

amadeus_api_key = os.getenv("AMADEUS_API_KEY")
amadeus_api_secret = os.getenv("AMADEUS_API_SECRET")

def get_amadeus_token():
    response = requests.post(
        'https://test.api.amadeus.com/v1/security/oauth2/token',
        data={
            'grant_type': 'client_credentials',
            'client_id': amadeus_api_key,
            'client_secret': amadeus_api_secret
        }
    )
    return response.json()["access_token"]

print(get_amadeus_token())
print(maps_api_key)
print(amadeus_api_key)


npJtqjNA0gwoCcLyJMG01CDUqFtf
AIzaSyA5Uao9hjBw9OudtZ5Sq-ru_j1EUsZ5cFMs
ip7U6tdqiwHWuuWsAvsC4MM8lficjGe7


In [31]:
gmaps = googlemaps.Client(key=maps_api_key)

locations = ["Vail, CO", "Beaver Creek, CO", "Breckenridge, CO", "Keystone, CO", "A-Basin, CO", "Eldora, CO", "Canyons, UT", "Heavenly, CA", "Northstar, CA", "Kirkwood, CA", "Afton Alps, MN", "Mt. Brighton, MI", "Verbier, Switzerland", "Ski Arlberg, Austria", "The 3 Valleys, France"]
nearest_airport = {
    "Vail, CO": "DEN",
    "Beaver Creek, CO": "DEN",
    "Breckenridge, CO": "DEN",
    "Keystone, CO": "DEN", 
    "A-Basin, CO": "DEN", 
    "Eldora, CO": "DEN", 
    "Canyons, UT": "SLC",
    "Heavenly, CA": "RNO",
    "Northstar, CA": "RNO",
    "Kirkwood, CA": "RNO",
    "Afton Alps, MN": "MSP",
    "Mt. Brighton, MI": "DTW",
    "Verbier, Switzerland": "GVA",
    "Ski Arlberg, Austria": "INNS",
    "The 3 Valleys, France": "GVA"
}


In [42]:
# STEP 1: BUILD COST MATRIX

def get_drive_info(origin, destination):
    try:
        res = gmaps.directions(origin, destination, mode="driving")
        print(f"Google Maps call: {origin} → {destination} → {bool(res)}")
        if res:
            leg = res[0]['legs'][0]
            distance_mi = leg['distance']['value'] * 0.000621371
            duration_hr = leg['duration']['value'] / 3600
            return distance_mi, duration_hr
    except Exception as e:
        print(f"Drive error: {origin} → {destination} → {e}")
    return None, None

def get_flight_cost_amadeus(orig_airport, dest_airport, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "originLocationCode": orig_airport,
        "destinationLocationCode": dest_airport,
        "departureDate": "2025-12-15",
        "adults": 1,
        "travelClass": "ECONOMY",
        "currencyCode": "USD",
        "max": 1
    }
    response = requests.get(
        "https://test.api.amadeus.com/v2/shopping/flight-offers",
        headers=headers,
        params=params
    )
    try:
        offers = response.json().get("data", [])
        if offers:
            return float(offers[0]["price"]["total"])
    except:
        pass
    return None

def build_cost_matrix(locations, nearest_airport, access_token):
    n = len(locations)
    cost_matrix = np.full((n, n), np.inf)
    mode_matrix = [['' for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            A, B = locations[i], locations[j]

            # DRIVE
            dist_drive, _ = get_drive_info(A, B)
            cost_drive = dist_drive * 0.5 if dist_drive else np.inf

            if dist_drive is not None and dist_drive < 150:
                cost_matrix[i][j] = cost_drive
                mode_matrix[i][j] = "drive"
                print(f"{A} → {B}: drive = ${cost_drive:.2f}, fly = skipped (short distance)")
                continue

            # FLY
            airport_A, airport_B = nearest_airport[A], nearest_airport[B]
            drive1, _ = get_drive_info(A, airport_A)
            drive2, _ = get_drive_info(airport_B, B)
            flight_price = get_flight_cost_amadeus(airport_A, airport_B, access_token)

            if None not in [drive1, drive2, flight_price]:
                cost_fly = flight_price + 0.5 * (drive1 + drive2)
            else:
                cost_fly = np.inf

            print(f"{A} → {B}: drive = ${cost_drive:.2f}, fly = ${cost_fly:.2f}")

            # CHOOSE BEST MODE
            if cost_drive < cost_fly:
                cost_matrix[i][j] = cost_drive
                mode_matrix[i][j] = "drive"
            else:
                cost_matrix[i][j] = cost_fly
                mode_matrix[i][j] = "fly"

            time.sleep(1)  # API rate limit buffer

    return cost_matrix, mode_matrix

In [45]:
token = get_amadeus_token()
flight_price = get_flight_cost_amadeus("GVA", "INN", token)
print(f"Flight from GVA to INN: ${flight_price}")

gmaps = googlemaps.Client(key=maps_api_key)
res = gmaps.directions("Vail, CO", "Beaver Creek, CO", mode="driving")
print(res)

Flight from GVA to INN: $230.9


ApiError: REQUEST_DENIED (The provided API key is invalid. )

In [43]:
# STEP 2: SOLVE TSP

def solve_tsp(cost_matrix):
    n = len(cost_matrix)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def cost_callback(from_idx, to_idx):
        from_node = manager.IndexToNode(from_idx)
        to_node = manager.IndexToNode(to_idx)
        return int(cost_matrix[from_node][to_node] * 100)

    transit_callback_idx = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_idx)

    params = pywrapcp.DefaultRoutingSearchParameters()
    params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(params)
    if not solution:
        raise Exception("TSP solution not found")

    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))
    return route

In [44]:
# STEP 3: EVALUATE TRIP

def summarize_trip(route, locations, cost_matrix, mode_matrix):
    total_cost = 0
    for i in range(len(route)-1):
        a, b = route[i], route[i+1]
        origin, dest = locations[a], locations[b]
        cost = cost_matrix[a][b]
        mode = mode_matrix[a][b]
        total_cost += cost
        print(f"{origin} → {dest}: ${cost:.2f} via {mode}")
    print(f"\nTotal trip cost: ${total_cost:.2f}")

# MAIN EXECUTION
locations = ["Vail, CO", "Beaver Creek, CO", "Breckenridge, CO", "Keystone, CO", "A-Basin, CO", "Eldora, CO", "Canyons, UT", "Heavenly, CA", "Northstar, CA", "Kirkwood, CA", "Afton Alps, MN", "Mt. Brighton, MI", "Verbier, Switzerland", "Ski Arlberg, Austria", "The 3 Valleys, France"]
nearest_airport = {
    "Vail, CO": "DEN",
    "Beaver Creek, CO": "DEN",
    "Breckenridge, CO": "DEN",
    "Keystone, CO": "DEN", 
    "A-Basin, CO": "DEN", 
    "Eldora, CO": "DEN", 
    "Canyons, UT": "SLC",
    "Heavenly, CA": "RNO",
    "Northstar, CA": "RNO",
    "Kirkwood, CA": "RNO",
    "Afton Alps, MN": "MSP",
    "Mt. Brighton, MI": "DTW",
    "Verbier, Switzerland": "GVA",
    "Ski Arlberg, Austria": "INN",
    "The 3 Valleys, France": "GVA"
}

token = get_amadeus_token()
cost_matrix, mode_matrix = build_cost_matrix(locations, nearest_airport, token)
route = solve_tsp(cost_matrix)
summarize_trip(route, locations, cost_matrix, mode_matrix)


Vail, CO → Beaver Creek, CO: drive = $inf, fly = $inf
Vail, CO → Breckenridge, CO: drive = $inf, fly = $inf
Vail, CO → Keystone, CO: drive = $inf, fly = $inf
Vail, CO → A-Basin, CO: drive = $inf, fly = $inf
Vail, CO → Eldora, CO: drive = $inf, fly = $inf
Vail, CO → Canyons, UT: drive = $inf, fly = $inf
Vail, CO → Heavenly, CA: drive = $inf, fly = $inf
Vail, CO → Northstar, CA: drive = $inf, fly = $inf
Vail, CO → Kirkwood, CA: drive = $inf, fly = $inf
Vail, CO → Afton Alps, MN: drive = $inf, fly = $inf
Vail, CO → Mt. Brighton, MI: drive = $inf, fly = $inf
Vail, CO → Verbier, Switzerland: drive = $inf, fly = $inf
Vail, CO → Ski Arlberg, Austria: drive = $inf, fly = $inf
Vail, CO → The 3 Valleys, France: drive = $inf, fly = $inf
Beaver Creek, CO → Vail, CO: drive = $inf, fly = $inf
Beaver Creek, CO → Breckenridge, CO: drive = $inf, fly = $inf
Beaver Creek, CO → Keystone, CO: drive = $inf, fly = $inf
Beaver Creek, CO → A-Basin, CO: drive = $inf, fly = $inf
Beaver Creek, CO → Eldora, CO: dr

KeyboardInterrupt: 

* Google Maps API Docs: https://developers.google.com/maps/documentation/directions/start

* FlightAPI.io Docs: https://www.flightapi.io/docs

* Google OR-Tools for Routing (TSP): https://developers.google.com/optimization/routing

* Haversine Formula (for backup distance calculations): https://en.wikipedia.org/wiki/Haversine_formula

* Python Requests Library: https://docs.python-requests.org/